In [14]:
!pip install tradingview_screener

In [15]:
from tradingview_screener import Query, Column
import pandas as pd
import datetime
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [16]:
class MarketScanner:
    def __init__(self, market='crypto', exchange='BINANCE', time_period=240):
        self.market = market
        self.exchange = exchange
        self.time_period = time_period

    def set_time_period(self, time_period):
        self.time_period = time_period

    def get_query(self):
        return Query().set_markets(self.market)

    def run_all(self):
        results = []
        methods = [method for method in dir(self) if callable(getattr(self, method)) and method.startswith('str')]
        for method in methods:
            result = getattr(self, method)()
            for index, row in result.iterrows():
                row_dict = row.to_dict()
                row_dict['source'] = method  # To determine which function the results come from
                results.append(row_dict)
        df = pd.DataFrame(results)
        if df.empty:
            return df
        df = df.groupby(['ticker','name', 'change', 'close'], as_index=False).agg({'source': ', '.join})
        df['source'] = df['source'].str.replace('str', 'S')
        df['Total_Strategy'] = df['source'].str.count('S')
        df = df.sort_values(by='Total_Strategy', ascending=False).reset_index(drop=True)
        return df
    
    def create_column(self, column_name):
        if self.time_period == "1D":
            return Column(column_name)
        else:
            return Column(f'{column_name}|{self.time_period}')
        
    def results(self, market, exchange):
        time_series = [120, 240, "1D", "1W"]
        results = []
        for time in time_series:
            scanner = MarketScanner(market=market, exchange=exchange)
            scanner.set_time_period(time)
            all_results_df = scanner.run_all()

            if all_results_df.empty:
                print(f"No results for time period {time}")
                continue

            all_results_df['Time_Period'] = time
            results.append(all_results_df)

        if results:
            combined_results = pd.concat(results, ignore_index=True)
            combined_results["date"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            combined_results = combined_results.sort_values(by=['Total_Strategy', 'name'], ascending=False).reset_index(drop=True)
            if market == "crypto":
                combined_results = combined_results[(combined_results['ticker'].str.contains('USDT.P'))]
            print(combined_results.to_string())
            return combined_results
        else:
            print("No results found for any time period.")
            return pd.DataFrame()
        
    def append_to_csv(self, market, exchange):
        new_data = self.results(market, exchange)
        file_path = f"{exchange}.csv"
        if os.path.exists(file_path):
            existing_data = pd.read_csv(file_path)
            existing_data = existing_data.dropna(axis=1, how='all')
            new_data_to_append = new_data[~new_data.set_index(['ticker', 'source', 'Time_Period']).index.isin(existing_data.set_index(['ticker', 'source', 'Time_Period']).index)]
            combined_data = pd.concat([existing_data, new_data_to_append], ignore_index=True)
        else:
            combined_data = new_data

        combined_data.to_csv(file_path, index=False)
        
    def back_testing(self, market, exchange):
        results = []
        file_name = f"{exchange}.csv"
        df = pd.read_csv(file_name)
        
        for name in df["name"]:
            data = (Query()
                    .set_markets(market)
                    .select('name', 'change', 'close')
                    .where(Column('exchange') == exchange, Column('name') == name)
                    .get_scanner_data())[1]
            results.append(data)

        if results:
            back_test = pd.concat(results, ignore_index=True)
            back_test["date"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            return back_test
        else:
            print("No results found.")
            return pd.DataFrame()
        
    def back_test_results(self, data_name, market, exchange):
        file_name = f"{exchange}.csv"
        past_df = pd.read_csv(file_name)
        now_df = self.back_testing(market, exchange)
        data_name = past_df.merge(now_df, on=["ticker","name"], suffixes=['_signal', '_now'])

        data_name['percent_change'] = ((data_name['close_now'] - data_name['close_signal']) / data_name['close_signal']) * 100
        data_name['date_signal'] = pd.to_datetime(data_name['date_signal'])
        data_name['date_now'] = pd.to_datetime(data_name['date_now'])
        data_name["time_diff"] = data_name["date_now"] - data_name["date_signal"]
        data_name.drop(["name", "Total_Strategy", "change_signal", "date_signal", "change_now", "date_now"], axis=1, inplace=True)
        data_name.drop_duplicates(subset=['ticker', 'source', 'Time_Period'], keep='first', inplace=True)

        return data_name
    
    def str_1_up_trend(self):
        query = self.get_query().select('name', 'change', 'close', 'volume', 'Perf.W').where(
            Column('exchange') == self.exchange,
            self.create_column('RSI') < 40.0,
            self.create_column('EMA5') < Column('close'),
            self.create_column('EMA20') < self.create_column('EMA5'),
            self.create_column('CCI20') < -80,
            self.create_column('ADX') > 15,
        ).limit(2000)
        return query.get_scanner_data()[1]

    def str_2_up_trend(self):
        query = self.get_query().select('name', 'change', 'close', 'volume', 'Perf.W').where(
            Column('exchange') == self.exchange,
            self.create_column('RSI') < 40.0,
            self.create_column('EMA5') < Column('close'),
            self.create_column('EMA20') < self.create_column('EMA5'),
            self.create_column('MACD.macd') > self.create_column('MACD.signal'),
            self.create_column('CCI20') < -80,
            self.create_column('ADX') > 15,
        ).limit(2000)
        return query.get_scanner_data()[1]


    def str_3_up_trend(self):
        query = self.get_query().select('name', 'change', 'close', 'volume', 'Perf.W').where(
            Column('exchange') == self.exchange,
            self.create_column('RSI') < 35.0,
            self.create_column('ADX') > 20,
            self.create_column('CCI20')< -80,
            self.create_column('Stoch.RSI.K').crosses_above(self.create_column('Stoch.RSI.D')),
            self.create_column('MACD.macd').crosses_above(self.create_column('MACD.signal'))
        ).limit(2000)
        return query.get_scanner_data()[1]

    def str_4_up_trend(self):
        query = self.get_query().select('name', 'change', 'close', 'volume', 'Perf.W').where(
            Column('exchange') == self.exchange,
            self.create_column('RSI') < 50,
            self.create_column('ADX') > 20,
            self.create_column('MACD.macd').crosses_above(self.create_column('MACD.signal')),
            self.create_column('CCI20')<-100,
            self.create_column('Stoch.RSI.K').crosses_above(self.create_column('Stoch.RSI.D')),
        ).limit(2000)
        return query.get_scanner_data()[1]

    def str_5_up_trend(self):
        query = self.get_query().select('name', 'change', 'close', 'volume', 'Perf.W').where(
            Column('exchange') == self.exchange,
            self.create_column('RSI') < 50,
            self.create_column('CCI20').crosses_above(-100),
            self.create_column('MACD.macd').crosses_above(self.create_column('MACD.signal')),
            self.create_column('ADX') > 20,
            self.create_column('Stoch.RSI.K').crosses_above(self.create_column('Stoch.RSI.D')),
            self.create_column('P.SAR')< Column('close'),
        ).limit(2000)
        return query.get_scanner_data()[1]

    def str_6_up_signal(self):
        query = self.get_query().select('name', 'change', 'close', 'volume', 'Perf.W').where(
            Column('exchange') == self.exchange,
                self.create_column('RSI') < 70,
                self.create_column('CCI20')< 60,
                self.create_column('MACD.macd').crosses_above(self.create_column('MACD.signal')),
                self.create_column('ADX') > 15,
                self.create_column('Stoch.RSI.K').crosses_above(self.create_column('Stoch.RSI.D')),
                #self.create_column('BB.lower')> Column('close'),
                #self.create_column('Aroon.Down').crosses_above(self.create_column('Aroon.Up')),

        ).limit(2000)
        return query.get_scanner_data()[1]


    def str_7_down_trend(self):
        query = self.get_query().select('name', 'change', 'close', 'volume', 'Perf.W').where(
            Column('exchange') == self.exchange,
            self.create_column('EMA5') > Column('close'),
            self.create_column('EMA20')> self.create_column('EMA5'),
            self.create_column('RSI') > 60.0,
            self.create_column('CCI20')>60,
            self.create_column('ADX') > 15,
        ).limit(2000)
        return query.get_scanner_data()[1]

    def str_8_down_trend(self):
        query = self.get_query().select('name', 'change', 'close', 'volume', 'Perf.W').where(
            Column('exchange') == self.exchange,
            self.create_column('EMA5') > Column('close'),
            self.create_column('EMA20')> self.create_column('EMA5'),
            self.create_column('CCI20')>65,
            self.create_column('RSI') > 60,
            self.create_column('MACD.macd').crosses_below(self.create_column('MACD.signal')),
            self.create_column('ADX') > 15,
        ).limit(2000)
        return query.get_scanner_data()[1]

    def str_9_down_trend(self):
        query = self.get_query().select('name', 'change', 'close', 'volume', 'Perf.W').where(
            Column('exchange') == self.exchange,
            self.create_column('ADX') > 20,
            self.create_column('CCI20')>70,
            self.create_column('MACD.macd').crosses_below(self.create_column('MACD.signal')),
            self.create_column('Stoch.RSI.K').crosses_below(self.create_column('Stoch.RSI.D')),
            self.create_column('RSI') > 50,
        ).limit(2000)
        return query.get_scanner_data()[1]

    def str_10_down_trend(self):
        query = self.get_query().select('name', 'change', 'close', 'volume', 'Perf.W').where(
            Column('exchange') == self.exchange,
            self.create_column('ADX') > 20,
            self.create_column('MACD.macd').crosses_below(self.create_column('MACD.signal')),
            self.create_column('Stoch.RSI.K').crosses_below(self.create_column('Stoch.RSI.D')),
            self.create_column('CCI20')>70,
            #self.create_column('P.SAR')> Column('close'),
            self.create_column('RSI') > 50,
        ).limit(2000)
        return query.get_scanner_data()[1]

    def str_11_down_trend(self):
        query = self.get_query().select('name', 'change', 'close', 'volume', 'Perf.W').where(
            Column('exchange') == self.exchange,
            #self.create_column('change') < 5.0,
            self.create_column('ADX') > 20,
            self.create_column('MACD.macd').crosses_below(self.create_column('MACD.signal')),
            #self.create_column('MACD.macd').crosses_below(0),
            self.create_column('Stoch.RSI.K').crosses_below(self.create_column('Stoch.RSI.D')),
            self.create_column('RSI') >55.0,
            self.create_column('P.SAR')> Column('close'),
            self.create_column('CCI20').crosses_below(100)
        ).limit(2000)
        return query.get_scanner_data()[1]

    def str_12_down_signal(self):
        query = self.get_query().select('name', 'change', 'close', 'volume', 'Perf.W').where(
            Column('exchange') == self.exchange,
                self.create_column('RSI') > 30.0,
                self.create_column('CCI20').between(-60,300),
                self.create_column('MACD.macd').crosses_below(self.create_column('MACD.signal')),
                self.create_column('ADX') >15,
                self.create_column('Stoch.RSI.K').crosses_below(self.create_column('Stoch.RSI.D')),
                #self.create_column('BB.upper')< Column('close')
                #self.create_column('Aroon.Up').crosses_below(self.create_column('Aroon.Down')),

        ).limit(2000)
        return query.get_scanner_data()[1]

In [17]:
scanner = MarketScanner()

In [18]:
scanner.append_to_csv(market='crypto', exchange='BINANCE')

                    ticker            name     change     close                                     source  Total_Strategy Time_Period                 date
6        BINANCE:DYMUSDT.P       DYMUSDT.P   7.561489  1.176400  S_3_up_trend, S_4_up_trend, S_6_up_signal               3          1D  2024-07-07 19:30:59
11       BINANCE:BELUSDT.P       BELUSDT.P  -1.952191  0.492200  S_3_up_trend, S_4_up_trend, S_6_up_signal               3          1D  2024-07-07 19:30:59
14       BINANCE:XVGUSDT.P       XVGUSDT.P  -4.415182  0.003702                S_4_up_trend, S_6_up_signal               2          1D  2024-07-07 19:30:59
16      BINANCE:SAGAUSDT.P      SAGAUSDT.P  24.134333  1.308500                S_5_up_trend, S_6_up_signal               2          1D  2024-07-07 19:30:59
25      BINANCE:NTRNUSDT.P      NTRNUSDT.P  -0.990874  0.379700                S_4_up_trend, S_6_up_signal               2          1D  2024-07-07 19:30:59
28     BINANCE:MAVIAUSDT.P     MAVIAUSDT.P   4.418891  1.443800 

In [19]:
scanner.append_to_csv(market='turkey', exchange='BIST')

        ticker   name    change       close                                             source  Total_Strategy Time_Period                 date
0   BIST:ULKER  ULKER -1.474759  173.700000  S_10_down_trend, S_12_down_signal, S_9_down_trend               3          1D  2024-07-07 19:31:30
1   BIST:TRGYO  TRGYO -0.558657   53.400002  S_10_down_trend, S_12_down_signal, S_9_down_trend               3         120  2024-07-07 19:31:30
2   BIST:ATAGY  ATAGY -1.845638   11.700000          S_3_up_trend, S_4_up_trend, S_6_up_signal               3         120  2024-07-07 19:31:30
3   BIST:ZEDUR  ZEDUR -1.067961   10.190000                        S_3_up_trend, S_6_up_signal               2         120  2024-07-07 19:31:30
4   BIST:VKING  VKING  0.883836   47.939999                                   S_12_down_signal               1         120  2024-07-07 19:31:30
5   BIST:VAKFN  VAKFN  0.000000    3.140000                                   S_12_down_signal               1          1W  2024-07-07 1

In [20]:
scanner.append_to_csv(market='america', exchange='NASDAQ')

          ticker   name     change     close                                             source  Total_Strategy Time_Period                 date
0    NASDAQ:PFTA   PFTA  -0.459982   10.8200  S_10_down_trend, S_12_down_signal, S_9_down_trend               3          1W  2024-07-07 19:32:15
1    NASDAQ:MCAA   MCAA  -0.085763   11.6500  S_10_down_trend, S_12_down_signal, S_9_down_trend               3          1D  2024-07-07 19:32:15
2    NASDAQ:BFRG   BFRG  -3.636364    1.5900          S_3_up_trend, S_4_up_trend, S_6_up_signal               3         120  2024-07-07 19:32:15
3     NASDAQ:BFI    BFI  -1.693064    0.1800          S_3_up_trend, S_4_up_trend, S_6_up_signal               3          1D  2024-07-07 19:32:15
4    NASDAQ:ASPS   ASPS   1.574803    1.2900          S_3_up_trend, S_4_up_trend, S_6_up_signal               3         240  2024-07-07 19:32:15
5    NASDAQ:ZPTA   ZPTA   1.229896    0.5350                        S_4_up_trend, S_6_up_signal               2         240  2024-

In [21]:
crypto_test = scanner.back_test_results(data_name='crypto_test', market='crypto', exchange='BINANCE')

In [22]:
turkey_test = scanner.back_test_results(data_name='turkey_test', market='turkey', exchange='BIST')

In [23]:
america_test = scanner.back_test_results(data_name='america_test', market='america', exchange='NASDAQ')

In [24]:
crypto_test

,ticker,close_signal,source,Time_Period,close_now,percent_change,time_diff
0,BINANCE:MAVIAUSDT.P,1.415800,"S_3_up_trend, S_4_up_trend, S_6_up_signal",1D,1.430600,1.045345,0 days 00:58:04
4,BINANCE:DYMUSDT.P,1.121700,"S_3_up_trend, S_4_up_trend, S_6_up_signal",1D,1.168000,4.127663,0 days 00:58:04
9,BINANCE:SAGAUSDT.P,1.307000,"S_5_up_trend, S_6_up_signal",1D,1.302500,-0.344300,0 days 00:58:04
10,BINANCE:NTRNUSDT.P,0.379400,"S_4_up_trend, S_6_up_signal",1D,0.380300,0.237217,0 days 00:58:04
13,BINANCE:DARUSDT.P,0.110500,"S_4_up_trend, S_6_up_signal",1D,0.110800,0.271493,0 days 00:58:04
16,BINANCE:IOUSDT.P,2.159000,S_12_down_signal,120,2.139000,-0.926355,0 days 00:58:04
18,BINANCE:DYMUSDT.P,1.120500,"S_4_up_trend, S_6_up_signal",1D,1.168000,4.239179,0 days 00:47:10
25,BINANCE:DYMUSDT.P,1.122500,S_3_up_trend,1D,1.168000,4.053452,0 days 00:47:10
30,BINANCE:MANTAUSDT.P,0.823700,"S_3_up_trend, S_4_up_trend, S_6_up_signal",1D,0.819100,-0.558456,0 days 00:20:28
31,BINANCE:BELUSDT.P,0.494400,"S_3_up_trend, S_4_up_trend, S_6_up_signal",1D,0.492400,-0.404531,0 days 00:20:28


In [25]:
turkey_test

,ticker,close_signal,source,Time_Period,close_now,percent_change,time_diff
0,BIST:ULKER,173.700000,"S_10_down_trend, S_12_down_signal, S_9_down_trend",1D,173.700000,0.0,0 days 00:58:25
1,BIST:TRGYO,53.400002,"S_10_down_trend, S_12_down_signal, S_9_down_trend",120,53.400002,0.0,0 days 00:58:25
6,BIST:ATAGY,11.700000,"S_3_up_trend, S_4_up_trend, S_6_up_signal",120,11.700000,0.0,0 days 00:58:25
11,BIST:ZEDUR,10.190000,"S_3_up_trend, S_6_up_signal",120,10.190000,0.0,0 days 00:58:25
16,BIST:VKING,47.939999,S_12_down_signal,120,47.939999,0.0,0 days 00:58:25
21,BIST:VAKFN,3.140000,S_12_down_signal,1W,3.140000,0.0,0 days 00:58:25
22,BIST:PENGD,7.750000,S_12_down_signal,1W,7.750000,0.0,0 days 00:58:25
23,BIST:ONRYT,68.199997,S_6_up_signal,120,68.199997,0.0,0 days 00:58:25
28,BIST:MEKAG,52.049999,S_6_up_signal,240,52.049999,0.0,0 days 00:58:25
33,BIST:LRSHO,3.840000,S_12_down_signal,1D,3.840000,0.0,0 days 00:58:25


In [26]:
america_test

,ticker,close_signal,source,Time_Period,close_now,percent_change,time_diff
0,NASDAQ:PFTA,10.8200,"S_10_down_trend, S_12_down_signal, S_9_down_trend",1W,10.8200,0.0,0 days 00:26:21
1,NASDAQ:MCAA,11.6500,"S_10_down_trend, S_12_down_signal, S_9_down_trend",1D,11.6500,0.0,0 days 00:26:21
2,NASDAQ:BFRG,1.5900,"S_3_up_trend, S_4_up_trend, S_6_up_signal",120,1.5900,0.0,0 days 00:26:21
6,NASDAQ:BFI,0.1800,"S_3_up_trend, S_4_up_trend, S_6_up_signal",1D,0.1800,0.0,0 days 00:26:21
7,NASDAQ:ASPS,1.2900,"S_3_up_trend, S_4_up_trend, S_6_up_signal",240,1.2900,0.0,0 days 00:26:21
11,NASDAQ:ZPTA,0.5350,"S_4_up_trend, S_6_up_signal",240,0.5350,0.0,0 days 00:26:21
15,NASDAQ:MARX,10.7700,"S_11_down_trend, S_12_down_signal",1W,10.7700,0.0,0 days 00:26:21
17,NASDAQ:WW,1.2800,S_6_up_signal,1D,1.2800,0.0,0 days 00:26:21
18,NASDAQ:VRCA,7.6300,S_6_up_signal,120,7.6300,0.0,0 days 00:26:21
22,NASDAQ:USEG,1.1500,S_6_up_signal,1W,1.1500,0.0,0 days 00:26:21
